In [2]:
import pandas as pd
import numpy as np

full_df = pd.read_csv("purchase_data.csv")

full_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


Getting Total Player Count

In [3]:
tot_play = len(full_df['SN'].unique())
print(tot_play)

576


Purchasing Analysis

In [7]:
count_items = len(full_df['Item Name'].unique())
avg_purch_price = round(full_df['Price'].mean(),2)
total_purch = len(full_df)
tot_rev = full_df['Price'].sum()

purch_df = pd.DataFrame({
    'Count of Unique Items':count_items,
    'Average Purchase Price':avg_purch_price,
    'Total Num. Purchases':total_purch,
    'Total Revenue':[tot_rev]
})
purch_df['Average Purchase Price'] = purch_df['Average Purchase Price'].map('${:.2f}'.format)
purch_df['Total Revenue'] = purch_df['Total Revenue'].map('${:.2f}'.format)
print('Purchasing Anlysis\n')
purch_df


Purchasing Anlysis



,Count of Unique Items,Average Purchase Price,Total Num. Purchases,Total Revenue
0,179,$3.05,780,$2379.77


Gender Demographics 

In [4]:
gender_grouped = full_df.groupby('Gender')

gender_df = pd.DataFrame(gender_grouped['Purchase ID'].count())
gender_df.rename(columns={
    'Purchase ID':'Count Of Purchases'
}, inplace=True)
#gender_df = gender_df.reset_index()
gender_df

,Count Of Purchases
Gender,
Female,113
Male,652
Other / Non-Disclosed,15


In [5]:
purch_per_gender = gender_grouped['Gender'].count()

gender_purch_percent = pd.DataFrame((purch_per_gender/total_purch)*100)
gender_purch_percent = gender_purch_percent.rename(columns={'Gender':'Percent of Purchases'})
#gender_purch_percent = gender_bug_percent.reset_index()
gender_purch_percent['Percent of Purchases'] = gender_purch_percent['Percent of Purchases'].map('{:.2f}%'.format)
gender_purch_percent

,Percent of Purchases
Gender,
Female,14.49%
Male,83.59%
Other / Non-Disclosed,1.92%


In [6]:
gender_full_df = gender_df.merge(gender_purch_percent, on='Gender')
gender_full_df.rename(columns={'Purchase ID':'Count of Purchases'}, inplace=True)
print('Gender Demographics\n')
gender_full_df

Gender Demographics



,Count Of Purchases,Percent of Purchases
Gender,,
Female,113,14.49%
Male,652,83.59%
Other / Non-Disclosed,15,1.92%


Purchase Anlysis By Gender

In [7]:
gender_avg_price = pd.DataFrame(gender_grouped['Price'].mean()) 
gend_tot_val = pd.DataFrame(gender_grouped['Price'].sum())
gender_count = pd.DataFrame(gender_grouped['SN'].nunique())

                            
gender_results =gender_df.merge(gender_avg_price.merge(gend_tot_val.merge(gender_count, on='Gender'), on='Gender'), on='Gender')
gender_results.rename(columns={
    'Purchase ID':'Purchase Count',
    'Price_x':'Average Purchase Price',
    'Price_y':'Total Purchase Value',
}, inplace=True)


gender_results['Avg. Purchase Total per Person'] = gender_results['Total Purchase Value'].astype(float)/ \
gender_results['SN'].astype(float)

gender_results['Average Purchase Price'] = gender_results['Average Purchase Price'].map('${:.2f}'.format)
gender_results['Total Purchase Value'] = gender_results['Total Purchase Value'].map('${:.2f}'.format)
gender_results['Avg. Purchase Total per Person'] = gender_results['Avg. Purchase Total per Person'].map('${:.2f}'.format)
gender_results.drop(columns=['SN'], inplace=True)
print("Purchase Analysis By Gender\n")
gender_results

Purchase Analysis By Gender



,Count Of Purchases,Average Purchase Price,Total Purchase Value,Avg. Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


Age Demographics

In [8]:
bins = [0,12,18,25,47]
labels = ['<12','12-18','18-24','>24']
full_df['Age Group'] = pd.cut(full_df['Age'], bins, labels=labels)
#full_df.head()

In [9]:
grouped_age = full_df.groupby('Age Group')
purch_count = pd.DataFrame(grouped_age['SN'].count())
avg_purch_price = pd.DataFrame(grouped_age['Price'].mean())
tot_purch_val = pd.DataFrame(grouped_age['Price'].sum())
people_count = pd.DataFrame(grouped_age['SN'].nunique())
age_df = purch_count.merge(avg_purch_price.merge(tot_purch_val.merge(people_count, on='Age Group'), on='Age Group'), on='Age Group')

age_df.rename(columns={
    'SN_x':'Purchase Count',
    'Price_x':'Average Purchase Price',
    'Price_y':'Total Purchase Value',
    'SN_y':'PCount'
}, inplace=True)

age_df['Avg. Purchase Total per Person'] = age_df['Total Purchase Value']/ age_df['PCount']
age_df.drop(columns=['PCount'], inplace=True)
age_df['Average Purchase Price'] = age_df['Average Purchase Price'].map('${:.2f}'.format)
age_df['Total Purchase Value'] = age_df['Total Purchase Value'].map('${:.2f}'.format)
age_df['Avg. Purchase Total per Person'] = age_df['Avg. Purchase Total per Person'].map('${:.2f}'.format)
print('Age Demographics\n')
age_df


Age Demographics



,Purchase Count,Average Purchase Price,Total Purchase Value,Avg. Purchase Total per Person
Age Group,,,,
<12,45,$3.19,$143.55,$4.22
12-18,119,$3.02,$359.27,$3.78
18-24,447,$3.06,$1365.94,$4.30
>24,169,$3.02,$511.01,$3.96


Top Spenders 

In [16]:
grouped_names = full_df.groupby('SN')

purch_count = pd.DataFrame(grouped_names['Purchase ID'].count())
purch_count.rename(columns={'SN':'Purchase Count'},inplace=True)

avg_purch_price = pd.DataFrame(grouped_names['Price'].mean())

tot_purch_val = pd.DataFrame(grouped_names['Price'].sum())

spenders_df = purch_count.merge(avg_purch_price.merge(tot_purch_val, on='SN'), on='SN')
spenders_df.rename(columns={'Price_x':'Average Purchase Price',
                           'Price_y':'Total Purchase Value'}, inplace=True)
spenders_df['Average Purchase Price'] = spenders_df['Average Purchase Price'].map('${:.2f}'.format)
#spenders_df['Total Purchase Value'] = spenders_df['Total Purchase Value'].map('${:.2f}'.format)
spenders_df = spenders_df.sort_values('Total Purchase Value', ascending=False)
spenders_df = spenders_df.iloc[0:5,:]
spenders_df['Total Purchase Value'] = spenders_df['Total Purchase Value'].map('${:.2f}'.format)
print('Top 5 Spenders\n')
display(spenders_df)


Top 5 Spenders



,Purchase ID,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


Top 5 Items 

In [18]:
grouped_id = full_df.groupby('Item ID')

id_count = pd.DataFrame(grouped_id['Item ID'].count())
id_count.rename(columns={'Item ID':'Purchase Count'},inplace=True)
#display(id_count.head())
avg_purch_price = pd.DataFrame(grouped_id['Price'].mean())
#display(avg_purch_price.head())
tot_purch_val = pd.DataFrame(grouped_id['Price'].sum())
#display(tot_purch_val.head())
names = pd.DataFrame(grouped_id['Item Name'].unique())

id_df = id_count.merge(avg_purch_price.merge(tot_purch_val.merge(names, on='Item ID'), on='Item ID'), on='Item ID')
id_df.rename(columns={'Price_x':'Average Purchase Price',
                           'Price_y':'Total Purchase Value'}, inplace=True)

id_df['Average Purchase Price'] = id_df['Average Purchase Price'].map('${:.2f}'.format)
#id_df['Total Purchase Value'] = id_df['Total Purchase Value'].map('${:.2f}'.format)

id_df_count = id_df.sort_values('Purchase Count', ascending=False)
id_df_count = id_df_count.iloc[0:5,:]
id_df_count = id_df_count[['Item Name','Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]
id_df_count['Total Purchase Value'] = id_df_count['Total Purchase Value'].map('${:.2f}'.format)

id_df_val = id_df.sort_values('Total Purchase Value', ascending=False)
id_df_val = id_df_val[['Item Name','Purchase Count', 'Average Purchase Price', 'Total Purchase Value']]
id_df_val = id_df_val.iloc[0:5,:]
id_df_val['Total Purchase Value'] = id_df_val['Total Purchase Value'].map('${:.2f}'.format)

print('Top 5 Items By Cont\n')
display(id_df_count)
print('\nTop 5 Items By Total Purchase Value\n')
display(id_df_val)

Top 5 Items By Cont



,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,,,,
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,$4.23,$50.76
145,[Fiery Glass Crusader],9,$4.58,$41.22
108,"[Extraction, Quickblade Of Trembling Hands]",9,$3.53,$31.77
82,[Nirvana],9,$4.90,$44.10
19,"[Pursuit, Cudgel of Necromancy]",8,$1.02,$8.16



Top 5 Items By Total Purchase Value



,Item Name,Purchase Count,Average Purchase Price,Total Purchase Value
Item ID,,,,
178,"[Oathbreaker, Last Hope of the Breaking Storm]",12,$4.23,$50.76
82,[Nirvana],9,$4.90,$44.10
145,[Fiery Glass Crusader],9,$4.58,$41.22
92,[Final Critic],8,$4.88,$39.04
103,[Singed Scalpel],8,$4.35,$34.80


Three Obseravble Trends:

    1) Based off of "Total Purchase Value" for the age group and "Percent of Purchases" for the gender; Males, age 18-24 would be the best demographic to target marketing towards.
    2) "Oathbreaker, Last Hope of the Breaking Storm" is easily the most popular item purchased within the game
    3) The Item, "Pursuite, Crudget of Necromancy", being ranked 5 with 8 purchases, shows that the audience who plays the game(and actually buys items), wont buy more of a lower quality item at a lower price.  Its price is $1.02 but all the more popular items are above $4.20
    4) As shown in the general Purchasing Analysis table, all of the data is only 780 purchases.  Trends predicted simply from observation on this data set could be misleading about purchasing behavior with a larger group of people playing the game